In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [16]:
# read in beerstyles csv
df = pd.read_csv('allbeerLinksRonessa.csv', encoding = "ISO-8859-1")

In [17]:
df.head()

,abv,beer_name,beer_style,beer_url,brewery,csv_name,match_name,num_ratings,score,style_url
0,5.00,Budweiser,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/29/65/,Anheuser-Busch,americanadjunctlager,americanadjunctlager,6308,2.36,https://www.beeradvocate.com/beer/style/38/
1,4.74,Pabst Blue Ribbon (PBR),American Adjunct Lager,https://www.beeradvocate.com/beer/profile/447/...,Pabst Brewing Company,americanadjunctlager,americanadjunctlager,5683,2.93,https://www.beeradvocate.com/beer/style/38/
2,4.60,Corona Extra,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/75/232/,Grupo Modelo S.A. de C.V.,americanadjunctlager,americanadjunctlager,5072,2.34,https://www.beeradvocate.com/beer/style/38/
3,4.60,Miller High Life,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/105/...,Miller Brewing Co.,americanadjunctlager,americanadjunctlager,3797,2.69,https://www.beeradvocate.com/beer/style/38/
4,4.70,Red Stripe Jamaican Lager,American Adjunct Lager,https://www.beeradvocate.com/beer/profile/221/...,Desnoes & Geddes Limited,americanadjunctlager,americanadjunctlager,3255,2.97,https://www.beeradvocate.com/beer/style/38/


In [18]:
# dfKristine = df.iloc[0:5000]
# dfRonessa = df.iloc[5000:10000]
# dfPaul = df.iloc[10000:15000]
# dfJing = df.iloc[15000:]

In [19]:
# dfJing.to_csv("JingBeer.csv")

In [20]:
# urls = [row['url'] for index,row in df.iterrows()]

In [21]:
# urls

#  beer_dict = {
#                     "beer_name": beer_list[i],
#                     "beer_style": style_names[x],
#                     "beer_url": "https://www.beeradvocate.com" + beer_links[i],
#                     "csv_name": csv_names[x],
#                     "match_name": match_names[x],
#                     "style_url": style_urls[x],
#                     "num_ratings": ratings_list[i],
#                     'score': scores[i],
#                     'abv': abv[i],
#                     'brewery': brewery[i]
#                 }

In [ ]:
Name = []
Style = []
Url = []
Score = [] #<span class="ba-ravg">2.36</span>
ScoreClass = []
RatingCount = []
Ranking = []
ReviewCount = []
pDev = [] #percent deviation
BrosScore = []
Brewer = []
Location = []
ABV = []
Availability = []
imageLink = []
csv_name = []
match_name = []
num_ratings = []

beerCount = 1

for index, row in df.iterrows():
    print(row[2])
    name = row['beer_name']
    style = row["beer_style"]
    url = row["beer_url"]
    
    Name.append(name)
    Style.append(style)
    Url.append(url)
    csv_name.append(row['csv_name'])
    match_name.append(row['match_name'])
    num_ratings.append(row['num_ratings'])
    Brewer.append(row['brewery'])
    ABV.append(row['abv'])
    Score.append(row['score'])
    
# test 1 url first
# url = "https://www.beeradvocate.com/beer/profile/29/65/"
# urls = [row['url'] for row in df.iterrows()]

# use this to test 2 urls in a row
# urls = ['https://www.beeradvocate.com/beer/profile/29/65/','https://www.beeradvocate.com/beer/profile/285/39766/']
# for url in urls:
    # go to each beer style page and get the top 50 beers (by count of ratings)
    # response from site
    try:
        print(beerCount)
        response = requests.get(url)

        # Create BeautifulSoup object; parse with 'html.parser'
        soup = BeautifulSoup(response.text, 'html.parser')

#         beerScore = soup.find('span', class_='ba-ravg')
#         if (beerScore.text is not None):
#             Score.append(beerScore.text)
#         else:
#             Score.append('')

        scoreclass = soup.find('div', {'id':'score_box'}).find_all('b')
        if (scoreclass[1] is not None):
            ScoreClass.append(scoreclass[1].text)
        else:
            ScoreClass.append('')

#         rating = soup.find('span', class_='ba-ratings')
#         if (rating.text is not None):
#             RatingCount.append(rating.text)
#         else:
#             RatingCount.append('')

        itemstats = soup.find('div', {'id': 'item_stats'}).find_all('dd')
        ranking = itemstats[0].text.replace("#", "").replace(",", "")
        if (ranking is not None):
            Ranking.append(ranking)
        else:
            Ranking.append('')

        reviews = itemstats[1].text.replace(",", "")
        if (reviews is not None):
            ReviewCount.append(reviews)
        else:
            ReviewCount('')

        pdev = re.sub('\s','',itemstats[3].text).replace("%", "")
        if (pdev is not None):
            pDev.append(pdev)
        else:
            pDev.append('')

        brosdiv = soup.find('span', {'class': 'ba-bro_score'})
        #print(brosdiv)
        bros = brosdiv.text.strip()
        if (bros is not None):
            BrosScore.append(bros)
        else:
            BrosScore.append('')

        # get brewer and brewery from infobox    
        infobox = soup.find('div', {'id': 'info_box'}).find_all('a')

#         brewer = infobox[0].find('b').text

#         if (brewer is not None):
#             Brewer.append(brewer)
#         else:
#             Brewer.append('')

        # what needs to be checked for empty here?
        # do i need to empty the list when i'm done using it for next time?
        location = []
        for ahref in infobox:
            if ('/place/' in ahref['href']):
                location.append(ahref.text)

                
        if (len(location) > 1):
            place = ', '.join(location)
        elif (len(location) == 1):
            place = location[0]
        else:
            place = ''
#         print(place)
        Location.append(place)

        # get ABV and availability from infoboxHeadings
        infoboxHeadings = soup.find('div', {'id': 'info_box'}).find_all('b')
#         abv = re.sub('\s','',infoboxHeadings[5].next_sibling).replace("%", "")
#         if (abv is not None):
#             ABV.append(abv)
#         else:
#             ABV.append('')

        availability = re.sub('\s','',infoboxHeadings[6].next_sibling)
        if (availability is not None):
            Availability.append(availability)
        else:
            Availability.append('')

        beerimages = soup.find('div', {'id': 'main_pic_norm'}).find_all('img')
        beerlink = beerimages[0]['src']
        if (beerlink is not None):
            imageLink.append(beerlink)
        else:
            imageLink.append('')

        beerCount += 1
    except:
        print('error!', beerCount, url)
        beerCount += 1 #move to the next one
    

American Adjunct Lager
1
American Adjunct Lager
2
American Adjunct Lager
3
American Adjunct Lager
4
American Adjunct Lager
5
American Adjunct Lager
6
American Adjunct Lager
7
American Adjunct Lager
8
American Adjunct Lager
9
American Adjunct Lager
10
American Adjunct Lager
11
American Adjunct Lager
12
American Adjunct Lager
13
American Adjunct Lager
14
American Adjunct Lager
15
American Adjunct Lager
16
American Adjunct Lager
17
American Adjunct Lager
18
American Adjunct Lager
19
American Adjunct Lager
20
American Adjunct Lager
21
American Adjunct Lager
22
American Adjunct Lager
23
American Adjunct Lager
24
American Adjunct Lager
25
American Adjunct Lager
26
American Adjunct Lager
27
American Adjunct Lager
28
American Adjunct Lager
29
American Adjunct Lager
30
American Adjunct Lager
31
American Adjunct Lager
32
American Adjunct Lager
33
American Adjunct Lager
34
American Adjunct Lager
35
American Adjunct Lager
36
American Adjunct Lager
37
American Adjunct Lager
38
American Adjunct Lage

American Adjunct Lager
309
American Adjunct Lager
310
American Adjunct Lager
311
American Adjunct Lager
312
American Adjunct Lager
313
American Adjunct Lager
314
American Adjunct Lager
315
American Adjunct Lager
316
American Adjunct Lager
317
American Adjunct Lager
318
American Adjunct Lager
319
American Adjunct Lager
320
American Adjunct Lager
321
American Adjunct Lager
322
American Adjunct Lager
323
American Adjunct Lager
324
American Adjunct Lager
325
American Adjunct Lager
326
American Adjunct Lager
327
American Adjunct Lager
328
American Adjunct Lager
329
American Adjunct Lager
330
American Adjunct Lager
331
American Adjunct Lager
332
American Adjunct Lager
333
American Adjunct Lager
334
American Adjunct Lager
335
American Adjunct Lager
336
American Adjunct Lager
337
American Adjunct Lager
338
American Adjunct Lager
339
American Adjunct Lager
340
American Adjunct Lager
341
American Adjunct Lager
342
American Adjunct Lager
343
American Adjunct Lager
344
American Adjunct Lager
345
A

Munich Helles Lager
635
Munich Helles Lager
636
Munich Helles Lager
637
Munich Helles Lager
638
Munich Helles Lager
639
Munich Helles Lager
640
Munich Helles Lager
641
Munich Helles Lager
642
Munich Helles Lager
643
Munich Helles Lager
644
Munich Helles Lager
645
Munich Helles Lager
646
Munich Helles Lager
647
Munich Helles Lager
648
Munich Helles Lager
649
Munich Helles Lager
650
Munich Helles Lager
651
Munich Helles Lager
652
Munich Helles Lager
653
Munich Helles Lager
654
Munich Helles Lager
655
Munich Helles Lager
656
Munich Helles Lager
657
Munich Helles Lager
658
Munich Helles Lager
659
Munich Helles Lager
660
Munich Helles Lager
661
Munich Helles Lager
662
Munich Helles Lager
663
Munich Helles Lager
664
Munich Helles Lager
665
Munich Helles Lager
666
Munich Helles Lager
667
Munich Helles Lager
668
Munich Helles Lager
669
Munich Helles Lager
670
Munich Helles Lager
671
Munich Helles Lager
672
Munich Helles Lager
673
Munich Helles Lager
674
Munich Helles Lager
675
Munich Helles La

German Pilsener
1034
German Pilsener
1035
German Pilsener
1036
German Pilsener
1037
German Pilsener
1038
German Pilsener
1039
German Pilsener
1040
German Pilsener
1041
German Pilsener
1042
German Pilsener
1043
German Pilsener
1044
German Pilsener
1045
German Pilsener
1046
German Pilsener
1047
German Pilsener
1048
German Pilsener
1049
German Pilsener
1050
German Pilsener
1051
German Pilsener
1052
German Pilsener
1053
German Pilsener
1054
German Pilsener
1055
German Pilsener
1056
German Pilsener
1057
German Pilsener
1058
German Pilsener
1059
German Pilsener
1060
German Pilsener
1061
German Pilsener
1062
German Pilsener
1063
German Pilsener
1064
German Pilsener
1065
German Pilsener
1066
German Pilsener
1067
German Pilsener
1068
German Pilsener
1069
German Pilsener
1070
German Pilsener
1071
German Pilsener
1072
German Pilsener
1073
German Pilsener
1074
German Pilsener
1075
German Pilsener
1076
German Pilsener
1077
German Pilsener
1078
German Pilsener
1079
German Pilsener
1080
German Pilsen

In [ ]:
# scoreclass = soup.find('div', {'id':'score_box'}).find_all('b')
# sc = scoreclass[1].text
# print (sc)
# print(imageLink)

In [13]:
num_ratings

[6308, 5683, 5072, 3797, 3255, 2967, 2924, 2766, 2494, 2316]

In [ ]:
featuredf = pd.DataFrame({"Name": Name, "URL": Url, "Style": Style, "Score":Score,"ScoreClass":ScoreClass,
                          "Ranking":Ranking,"ReviewCount":ReviewCount,"pDev":pDev,
                         "BrosScore":BrosScore,"Brewer":Brewer,"Location":Location,"ABV":ABV,
                         "Availaility":Availability,"ImageLink":imageLink, "csv_name": csv_name, "match_name": match_name,
                         "num_ratings": num_ratings})

In [ ]:
featuredf

In [ ]:
# featuredf['Ranking'] = featuredf['Ranking'].str.replace(",", "")
# featuredf['RatingCount'] = featuredf['RatingCount'].str.replace(",", "")
# featuredf['ReviewCount'] = featuredf['ReviewCount'].str.replace(",", "")

In [ ]:
# save to csv
featuredf.to_csv('AllScrapedBeerFeatures.csv', index=False)

In [26]:
# read in beerstyles csv
test = pd.read_csv('AllBeersEachStyle.csv', encoding = "ISO-8859-1")

In [29]:
test['urlHack'] = test['url'] + ' '

In [30]:
test.to_csv('AllBeersEachStyleHack.csv')

In [34]:
featuredf['beerid'] = featuredf['ImageLink'].replace(['https://cdn.beeradvocate.com/im/beers/',',jpg'],'', inplace=True)
featuredf

,ABV,Availaility,Brewer,BrosScore,ImageLink,Location,Ranking,RatingCount,ReviewCount,Score,ScoreClass,pDev,beerid
0,5.60,Year-round,UNICER União Cervejeira S.A.,3.18,https://cdn.beeradvocate.com/im/beers/1295.jpg,Portugal,"45,086",372,153,2.81,Poor,21.71,None
1,5.00,Year-round,Commonwealth Brewery Ltd.,0,https://cdn.beeradvocate.com/im/beers/3050.jpg,Bahamas,"45,189",337,126,2.65,Poor,23.4,None
2,4.90,Year-round,"Carlton & United Breweries, Ltd.",0,https://cdn.beeradvocate.com/im/beers/1541.jpg,Australia,"45,285",328,149,2.41,Poor,32.78,None
3,4.50,Year-round,World Brews,0,https://cdn.beeradvocate.com/im/beers/64297.jpg,"California, United States","45,239",317,154,2.54,Poor,29.53,None
4,5.90,Year-round,Anheuser-Busch,0,https://cdn.beeradvocate.com/im/beers/3474.jpg,"Missouri, United States","45,345",317,66,1.75,Awful,39.43,None
5,4.50,Year-round,Stevens Point Brewery,0,https://cdn.beeradvocate.com/im/beers/2811.jpg,"Wisconsin, United States","44,703",317,140,3.08,Okay,19.48,None
6,5.00,Year-round,Coors Brewing Company (Molson-Coors),3.3,https://cdn.beeradvocate.com/im/beers/1466.jpg,"Colorado, United States","45,244",302,99,2.52,Poor,30.95,None
7,4.78,Year-round,Pabst Brewing Company,0,https://cdn.beeradvocate.com/im/beers/3906.jpg,"California, United States","45,219",286,109,2.58,Poor,27.52,None
8,4.60,Year-round,Rainier Brewing Co.,0,https://cdn.beeradvocate.com/im/beers/1720.jpg,"Texas, United States","45,132",280,86,2.72,Poor,26.1,None
9,4.30,Year-round,Straub Brewery,0,https://cdn.beeradvocate.com/im/beers/2832.jpg,"Pennsylvania, United States","44,334",271,116,3.2,Okay,17.5,None
